In [1]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer

C:\Users\gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\utils\_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
df = pd.read_csv("fashion.csv")
df.head()

,ProductId,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle,Image,ImageURL
0,42419,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top,42419.jpg,http://assets.myntassets.com/v1/images/style/p...
1,34009,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top,34009.jpg,http://assets.myntassets.com/v1/images/style/p...
2,40143,Girls,Apparel,Topwear,Tops,Blue,Casual,Gini and Jony Girls Pretty Blossom Blue Top,40143.jpg,http://assets.myntassets.com/v1/images/style/p...
3,23623,Girls,Apparel,Topwear,Tops,Pink,Casual,Doodle Kids Girls Pink I love Shopping Top,23623.jpg,http://assets.myntassets.com/v1/images/style/p...
4,47154,Girls,Apparel,Bottomwear,Capris,Black,Casual,Gini and Jony Girls Black Capris,47154.jpg,http://assets.myntassets.com/v1/images/style/p...


In [19]:

df=df[['ProductId','Gender','Category','SubCategory','ProductType','Colour','Usage','ProductTitle']]
df['CombinedString'] = df.apply(lambda row: ' '.join([f"{col}: {row[col]}," for col in df.columns]), axis=1)
df.head(4)

,ProductId,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle,CombinedString
0,42419,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top,"ProductId: 42419, Gender: Girls, Category: App..."
1,34009,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top,"ProductId: 34009, Gender: Girls, Category: App..."
2,40143,Girls,Apparel,Topwear,Tops,Blue,Casual,Gini and Jony Girls Pretty Blossom Blue Top,"ProductId: 40143, Gender: Girls, Category: App..."
3,23623,Girls,Apparel,Topwear,Tops,Pink,Casual,Doodle Kids Girls Pink I love Shopping Top,"ProductId: 23623, Gender: Girls, Category: App..."


In [ ]:
# Load T5 model and tokenizer
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to generate descriptions using T5 model
def generate_description(text):
    input_text = f"summarize: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)
    output_ids = model.generate(input_ids, max_length=150, num_beams=2, length_penalty=0.7, early_stopping=True)
    description = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return description

# # Apply the function to generate descriptions for each row
df['Description'] = df.apply(lambda row: generate_description(f"{row['CombinedString']}"), axis=1)

# # Save the DataFrame with generated descriptions to a new CSV file
df['Description'].head(3)
df.to_csv('fashion_descriptions.csv', index=False)

In [10]:
df["CombinedString"][1]

'ProductId: 34009, Gender: Girls, Category: Apparel, SubCategory: Topwear, ProductType: Tops, Colour: Black, Usage: Casual, ProductTitle: Gini and Jony Girls Black Top, Image: 34009.jpg, ImageURL: http://assets.myntassets.com/v1/images/style/properties/dce310e4c15223a6c964631190263284_images.jpg,'